In [ ]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta
import pyarrow

In [47]:
# Adapter Layer

def read_csv_to_df(bucket, objects):
    csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8') #Revisar este
    data = StringIO(csv_obj_init)
    df_init = pd.read_csv(data, delimiter=',')

    df_all = pd.DataFrame(columns=df_init.columns)

    for obj in objects:
        csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
        data = StringIO(csv_obj)
        df = pd.read_csv(data, delimiter=',')
        df_all = pd.concat([df,df_all], ignore_index=True)
    
    return df_all

def write_df_to_s3(df_all, key, bucket_target):
    
    
    out_buffer = BytesIO()
    
    #Create a .parquet file
    df_all.to_parquet(out_buffer, index=False)
    
    #Upload the file to the bucket with the key and the .parquet file stored in the buffer
    bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)
    pass

def return_objects(bucket, arg_date_dt):
    
    #Get all the objects according to the condition given and return them
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]
    
    return objects

In [57]:
# Application Layer

def extract(bucket, args):
    #Get the objects 
    objects = return_objects(bucket, args)
    
    #Get the dataframe out of the csv files
    df_all = read_csv_to_df(bucket, objects)
    
    return df_all

def transform_report(df_all, arg_date):
    
    df_all.dropna(inplace=True)
    df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')
    df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
    df_all.drop(columns=['prev_closing_price'], inplace=True)
    df_all = df_all.round(decimals=2)
    df_all.reset_index(inplace=True)
    df_all = df_all[df_all.Date >= arg_date] 
    
    return df_all

def load(df_all, bucket_target):
    #Generate a key to save the dataframe
    key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
    
    #Write the file in the cloud
    write_df_to_s3(df_all, key, bucket_target)
    pass

def etl_report(bucket_target, key):
    for obj in bucket_target.objects.all():
        print(obj.key)
    prq_obj = bucket_target.Object(key=key).get().get('Body').read()
    data = BytesIO(prq_obj)
    df_report = pd.read_parquet(data)
    
    return df_report
    

In [65]:
# main function entrypoint

def main():
    #Configuración de la conexión con AWS
    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('xetra-1234')
    
    bucket_target = s3.Bucket('xetra-ajlj')

    arg_date = '2022-12-31'
    arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)
    
    key = 'xetra_daily_report_20230224_235235.parquet'
    
    # ETL
    df_all = extract(bucket, arg_date_dt)
    df_transformed = transform_report(df_all, arg_date)
    load(df_transformed, bucket_target)
    report = etl_report(bucket_target, key)
    print(report)
    

In [66]:
# run

main()

xetra_daily_report_20230217_225703.parquet
xetra_daily_report_20230223_223956.parquet
      index          ISIN        Date  opening_price_eur  closing_price_eur  \
0         1  AT000000STR1  2022-12-31              36.60              36.70   
1         3  AT00000FACC2  2022-12-31               8.05               8.57   
2         5  AT0000606306  2022-12-31              14.51              15.00   
3         7  AT0000609607  2022-12-31              11.74              12.06   
4         9  AT0000644505  2022-12-31              98.20              99.20   
...     ...           ...         ...                ...                ...   
3227   6455  XS2284324667  2022-12-31              39.48              38.92   
3228   6457  XS2314659447  2022-12-31               8.87               8.80   
3229   6459  XS2314660700  2022-12-31              22.26              21.92   
3230   6461  XS2376095068  2022-12-31              34.29              36.50   
3231   6463  XS2434891219  2022-12-31        

## Reading the uploaded file

In [ ]:
trg_bucket = 'xetra-bucket-12345'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20220310_110626.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [ ]:
df_report